## Nikolaos Papadopoulos

#### Course: Large Scale Data Management



In [1]:
#Imports
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import os
import sys
import re

In [2]:
#Adding the below configurations to avoid errors
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

#Build session
spark = SparkSession.builder.appName("zillow").getOrCreate()

In [3]:
#Apache Arrow/Most beneficial to Python users that work with Pandas/NumPy data
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [4]:
#Reading data
df = spark.read.csv(r'zillow.csv',header=True)

In [5]:
#First glance over data
df.show(20)

+--------------+-------+------------+-----+-----------+----------+--------------------+--------------------+--------------------+
|         title|address|        city|state|postal_code|     price|  facts and features|real estate provider|                 url|
+--------------+-------+------------+-----+-----------+----------+--------------------+--------------------+--------------------+
|Condo for sale|   null|  Somerville|   MA|      02145|  $342,000|2 bds, 1.0 ba ,70...|William Raveis R....|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|      02116|$1,700,000|2 bds, 2.0 ba ,12...|Century 21 North ...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|      02118|  $336,500|1 bds, 1.0 ba ,10...|Maloney Propertie...|https://www.zillo...|
|House for sale|   null|      Boston|   MA|      02118|$9,950,000|4 bds, 7.0 ba ,68...|Campion & Company...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|      02128|  $479,000|2 bds, 3.0 ba ,10...|Ber

In [6]:
#Extract number of bedrooms.
fn =  F.udf(lambda x: x.split(',')[0].split(' ')[0])
df = df.withColumn('beds',fn('facts and features'))

#Extract number of bathrooms. 
fn2 = F.udf(lambda x: x.split(',')[1].split(' ')[1])
df = df.withColumn('baths',fn2('facts and features'))

#Extract sqft.
fn3 = F.udf(lambda x:x.split(',')[2].split(' ')[0])
df = df.withColumn('sqft',fn3('facts and features'))

In [7]:
#Replacing Nan Values
df = df.replace('None','0')

#Transforming strings into integers
fn4 = F.udf(lambda x: int(float(x)))
df = df.withColumn('baths',fn4('baths'))
df = df.withColumn('beds',fn4('beds'))
df = df.withColumn('sqft',fn4('sqft'))

In [8]:
#Extract type. 

fn5 = F.udf(lambda x:'Condo' if 'condo' in x.lower() else('House' if 'house' in x.lower() else('Multi-Family-Home' if 'home' in x.lower() else ('Lot/Land' if 'land' in x.lower() else 'House' ))))
df = df.withColumn('type',fn5('title'))

In [9]:
#Extract offer.This can be `sale`, `rent`, `sold`, `forclose`.

fn6 = F.udf(lambda x:'Foreclose' if x == "Foreclosure" else ('Sold' if 'sold' in x.lower() else ('Rent' if 'rent' in x.lower() else ('Sale' if 'sale' in x.lower() else 'Other'))))
df = df.withColumn('offer',fn6('title'))

In [10]:
#Filter out listings that are not for sale.
df = df[df.offer=='Sale']

In [11]:
#Extract price. You will implement a UDF that processes the `price` column and extract the price.
#Prices are stored as strings in the CSV. This UDF parses the string and returns the price as an integer.

fn9 = F.udf(lambda x: int(re.sub("[^0-9]", "",x)))
df = df.withColumn('prices',fn9('price'))

In [12]:
#Filter out listings with more than 10 bedrooms

df = df[df.beds<=10]

In [13]:
#Filter out listings with price greater than 20000000 and lower than 100000

df = df[(df.prices<20000000) & (df.prices >100000)]

In [14]:
#Filter out listings that are not houses

df = df[df.type=="House"]

In [15]:
#Calculate average price per sqft for houses for sale grouping them by the number of bedrooms.
#Sometimes an error pops up -> Re-run cell.

df.groupBy("beds").agg(F.avg(df.prices/df.sqft)).show()

+----+--------------------+
|beds|avg((prices / sqft))|
+----+--------------------+
|   7|  1126.0252348993286|
|   3|   687.2172712125251|
|   8|   1567.647058823529|
|   0|              1250.0|
|   5|   733.9532477532824|
|   6|   422.3111656297147|
|   9|  1108.1412183984849|
|   4|   909.1473996440552|
|   2|   716.0259107836451|
+----+--------------------+

